# 電流源推定(Source Estimation)    
https://neuroimage.usc.edu/brainstorm/Tutorials/SourceEstimation?highlight=%28source%29

In [1]:
import matlab
import scipy 
from scipy import io
import pandas as pd
from pandas import DataFrame, Series
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import chainer.functions as F
import chainer.links as L
from chainer import Variable, optimizers, Chain, cuda
#import cupy as xp
import math
import datetime

from sklearn import preprocessing

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## ちょっとGPUを見る

*bash*でやる 

```sh:
    lspci | grep -i nvidia   
```

06:00.0 3D controller: NVIDIA Corporation GK210GL [Tesla K80] (rev a1)
07:00.0 3D controller: NVIDIA Corporation GK210GL [Tesla K80] (rev a1)

*bash*でやる 

```sh:
    lspci | grep -i nvidia   
```

06:00.0 3D controller: NVIDIA Corporation GK210GL [Tesla K80] (rev a1)
07:00.0 3D controller: NVIDIA Corporation GK210GL [Tesla K80] (rev a1)

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## data_table作成  
[ID, PATH, ver,original_ID, label, (result_parh)]

In [46]:
#フォルダ名, PATHを取得(140個)
folder_name = [os.path.basename(r) for r in glob.glob('/work/project/MEG_ChildrenBiomarker/data/S*')]
list_folder = glob.glob('/work/project/MEG_ChildrenBiomarker/data/S*')
oriID = [folder_name[i][2:-2] for i in range(0, 140)]

In [47]:
list_folder = [list_folder[i] + '/1_SSP' for i in range(0, 140)]

In [48]:
data_table = pd.DataFrame({'PATH':list_folder, 'ID':folder_name, 'Ori_ID':oriID})

In [49]:
data_table.loc[data_table['ID'].str.contains('v1'), 'Version'] = 1
data_table.loc[data_table['ID'].str.contains('v2'), 'Version'] = 2

In [50]:
data_table.loc[data_table['ID'].str.contains('TD'), 'label'] = 0
data_table.loc[data_table['ID'].str.contains('CL'), 'label'] = 1

In [51]:
#性別　男:1, 女:2
data_table['sex'] = 1
data_table.loc[data_table['ID'].str.contains('RE'), 'sex'] = 2

In [53]:
data_table.to_csv('/work/project/MEG_ChildrenBiomarker/data/data_table.csv', index=False)

In [2]:
Data_Table = pd.read_csv('data_table.csv')
Data_Table.head(140)

,PATH,ID,Ori_ID,Version,label,sex,Result_PATH
0,/work/project/MEG_ChildrenBiomarker/data/S2TD0...,S2TD0036v1,TD0036,1.0,0.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
1,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0216v1,CL0216,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
2,/work/project/MEG_ChildrenBiomarker/data/S1TD0...,S1TD0032v1,TD0032,1.0,0.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
3,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0044v1,CL0044,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
4,/work/project/MEG_ChildrenBiomarker/data/S1CL0...,S1CL0045v1,CL0045,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
5,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0008v1,CL0008,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
6,/work/project/MEG_ChildrenBiomarker/data/S2TD0...,S2TD0056v1,TD0056,1.0,0.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
7,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0138v1,CL0138,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
8,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0177v2,CL0177,2.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
9,/work/project/MEG_ChildrenBiomarker/data/S2TD0...,S2TD0006v1,TD0006,1.0,0.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...


In [13]:
Data_Table['label'].value_counts()

1.0    79
0.0    57
Name: label, dtype: int64

## 解析用の関数を作成

In [ ]:
# test = []
def  function(PATH):
    list_dir = os.listdir(str(PATH)) 
    list_dir = [s for s in list_dir if ('results_MN' in s) or ('data_block' in s)] 
    
    #matファイルの読み込み
    #data_block = io.loadmat(PATH + '/' + list_dir[0] ,squeeze_me = True)
    #results_MN_MEG_KERNEL = io.loadmat(PATH + '/' + list_dir[1], squeeze_me = True)
    data_block = [s for s in list_dir if 'data_block' in s]
    data_block = io.loadmat(PATH + '/' + data_block[0] ,squeeze_me = True)
    results_MN_MEG_KERNEL = [n for n in list_dir if 'results' in n]
    results_MN_MEG_KERNEL = io.loadmat(PATH + '/' + results_MN_MEG_KERNEL[0] ,squeeze_me = True)
    
    #変数の読み込み
    F = data_block["F"]
    Imaging_Kernel = results_MN_MEG_KERNEL["ImagingKernel"]
    Good_Channel = results_MN_MEG_KERNEL["GoodChannel"]
    
    #Fを間引く(F(Channel * 18000))
    F = F[Good_Channel.T, :]
    
    #Imaging_Kernelを150個に分割しておく(順に取り除く) (0~149個)
    #nRandom_Kernel= [Imaging_Kernel[100*j -99:j*100+1, :] for j in range(1,151)]
    
    #Imaging_Kernelを150個に分割しておく(無作為抽出)
    np.random.seed(12)
    row = np.random.randint(1, 15002, 15000)
    row = row.reshape(100, 150)
    row.shape 
    Random_Kernel= [Imaging_Kernel[row[:, k], :] for k in range(0,150)]
    
    #電流源推定
    Current = [xp.dot(Random_Kernel[l], F, out=None) for l in range(0,150)]
    Current = [xp.asnumpy(Current[i]) for i in range(0, 150)]
    #nCurrent = [np.dot(nRandom_Kernel[m], F) for m in range(0,150)]
        
    #やらかしたのでPATHから逆算してIDをとる
    corr_PATH = np.where(Data_Table == PATH)
    tuple_list = [n for n in corr_PATH]
    a = tuple_list[0][0]
        
    #患者ごとのフォルダに保存する
    #os.mkdir('/work/project/MEG_ChildrenBiomarker/SourceEstimate/' + Data_Table.ID[a])
    for n in range(0,150):
        np.save('/work/project/MEG_ChildrenBiomarker/SourceEstimate/' + Data_Table.ID[a] +'/random_%d' %n,Current[n])
        #np.save('/work/project/MEG_ChildrenBiomarker/SourceEstimate/' + Data_Table.ID[a] +'/nrandom_%d' %n,nCurrent[n])
        
    return print('Data_Table.ID[%d]'%a)

In [ ]:
#48, 83, 138のGoogChannel #119はぞんざい
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ[" CUDA_VISIBLE_DEVICES "] = "0"
for i in range(0,140):
    function(Data_Table.PATH[i])

In [55]:
result_path = [os.path.basename(r) for r in glob.glob('/work/project/MEG_ChildrenBiomarker/SourceEstimate/S*')]

In [56]:
Result_PATH = ['/work/project/MEG_ChildrenBiomarker/SourceEstimate/'+ result_path[i] for i in range(0,140)] 
Data_Table['Result_PATH'] = Result_PATH

In [57]:
#テーブルの保存  
Data_Table.to_csv('/work/project/MEG_ChildrenBiomarker/data/data_table.csv', index=False)

In [58]:
#うまくいかなかったものを消しておく#48, 83, 138のGoogChannel #119はぞんざい
Data_Table.drop(index= [48, 83, 138, 119], inplace = True)
Data_Table.to_csv('/work/project/MEG_ChildrenBiomarker/code/data_table.csv', index=False)

In [59]:
Data_Table = pd.read_csv('/work/project/MEG_ChildrenBiomarker/code/data_table.csv')

In [65]:
Data_Table

,PATH,ID,Ori_ID,Version,label,sex,Result_PATH
0,/work/project/MEG_ChildrenBiomarker/data/S2TD0...,S2TD0036v1,TD0036,1.0,0.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
1,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0216v1,CL0216,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
2,/work/project/MEG_ChildrenBiomarker/data/S1TD0...,S1TD0032v1,TD0032,1.0,0.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
3,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0044v1,CL0044,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
4,/work/project/MEG_ChildrenBiomarker/data/S1CL0...,S1CL0045v1,CL0045,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
5,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0008v1,CL0008,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
6,/work/project/MEG_ChildrenBiomarker/data/S2TD0...,S2TD0056v1,TD0056,1.0,0.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
7,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0138v1,CL0138,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
8,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0177v2,CL0177,2.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
9,/work/project/MEG_ChildrenBiomarker/data/S2TD0...,S2TD0006v1,TD0006,1.0,0.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...


In [73]:
#os.mkdir('/work/project/MEG_ChildrenBiomarker/train_model')
def cutting(sample_path):
    
    #探す
    data = sample_path + '/random_0.npy' 
    #読み込む
    ar_1 = np.load(data)
    #100×180の100個分にする
    ar_2 = [ar_1[:, j*180:(j+1)*180] for j in range(0,100)]
    #正則化しておく
    mms = preprocessing.MinMaxScaler()
    ar_3 = [mms.fit_transform(ar_2[k]) for k in range(0,100)]
    
    #やらかしたのでPATHから逆算してIDをとる
    corr_PATH = np.where(Data_Table == sample_path)
    tuple_list = [n for n in corr_PATH]
    a = tuple_list[0][0]
    
    for l in range(0,100):
        np.save('/work/project/MEG_ChildrenBiomarker/train_model/' + Data_Table.ID[a] +'_%d'%l, ar_3[l])
        
    return print('Data_Table.ID[%d]'%a)

In [74]:
#読み込んだarrayをぶっこむ #100*180000
sample = [cutting(Data_Table.Result_PATH[i]) for i in range(0, 136)]

Data_Table.ID[0]
Data_Table.ID[1]
Data_Table.ID[2]
Data_Table.ID[3]
Data_Table.ID[4]
Data_Table.ID[5]
Data_Table.ID[6]
Data_Table.ID[7]
Data_Table.ID[8]
Data_Table.ID[9]
Data_Table.ID[10]
Data_Table.ID[11]
Data_Table.ID[12]
Data_Table.ID[13]
Data_Table.ID[14]
Data_Table.ID[15]
Data_Table.ID[16]
Data_Table.ID[17]
Data_Table.ID[18]
Data_Table.ID[19]
Data_Table.ID[20]
Data_Table.ID[21]
Data_Table.ID[22]
Data_Table.ID[23]
Data_Table.ID[24]
Data_Table.ID[25]
Data_Table.ID[26]
Data_Table.ID[27]
Data_Table.ID[28]
Data_Table.ID[29]
Data_Table.ID[30]
Data_Table.ID[31]
Data_Table.ID[32]
Data_Table.ID[33]
Data_Table.ID[34]
Data_Table.ID[35]
Data_Table.ID[36]
Data_Table.ID[37]
Data_Table.ID[38]
Data_Table.ID[39]
Data_Table.ID[40]
Data_Table.ID[41]
Data_Table.ID[42]
Data_Table.ID[43]
Data_Table.ID[44]
Data_Table.ID[45]
Data_Table.ID[46]
Data_Table.ID[47]
Data_Table.ID[48]
Data_Table.ID[49]
Data_Table.ID[50]
Data_Table.ID[51]
Data_Table.ID[52]
Data_Table.ID[53]
Data_Table.ID[54]
Data_Table.ID[55]
Da

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [87]:
#labelは逆算して作る
label = [os.path.basename(r) for r in glob.glob('/work/project/MEG_ChildrenBiomarker/train_model/S*')]
train_label = [0 if 'TD' in s else 1 for s in label]
train_label = np.array(train_label)
np.save('/work/project/MEG_ChildrenBiomarker/train_model/train_label', train_label )
#data_table.loc[data_table['ID'].str.contains('TD'), 'label'] = 0
#data_table.loc[data_table['ID'].str.contains('CL'), 'label'] = 1